# GPU

This simple appliance allows you to easily reserve a bare metal node with a GPU accelerator (availability permitting).

## Create experiment container

This container provides the following:

- One node with an attached GPU
- One Floating IP

### Configuration

In [ ]:
OS_PROJECT_NAME="$OS_PROJECT_NAME"
OS_REGION_NAME="CHI@TACC" # Currently only CHI@TACC has GPU nodes
IMAGE=CC-CentOS7-CUDA10
NODE_COUNT=1

# A unique name for most provisioned resources to avoid collisions
RESOURCE_NAME="${USER}-gpu-$(date +%b%d)"

### Reservation

In [ ]:
lease_name="$RESOURCE_NAME"
network_name="$RESOURCE_NAME"
public_network_id=$(openstack network show public -f value -c id)

blazar lease-create \
  --physical-reservation min="$NODE_COUNT",max="$NODE_COUNT",resource_properties="[\"=\", \"\$gpu.gpu\", \"True\"]" \
  --reservation resource_type=virtual:floatingip,network_id="$public_network_id",amount=1 \
  --start-date "$(date +'%Y-%m-%d %H:%M')" \
  --end-date "$(date +'%Y-%m-%d %H:%M' -d'+1 day')" \
  "$lease_name"

# Wait for lease to start
timeout 300 bash -c 'until [[ $(blazar lease-show $0 -f value -c status) == "ACTIVE" ]]; do sleep 1; done' "$lease_name" \
    && echo "Lease started successfully!"

#
# Fetch information about which resources were reserved for later use
#

reservations=$(blazar lease-show "$lease_name" -f json \
  | jq -r '.reservations')
host_reservation_id=$(jq -rs 'map(select(.resource_type=="physical:host"))[].id' <<<"$reservations")
fip_reservation_id=$(jq -rs 'map(select(.resource_type=="virtual:floatingip"))[].id' <<<"$reservations")

fip=$(openstack floating ip list --tags "reservation:$fip_reservation_id" -f json)
fip_id=$(jq -r 'map(.ID)[0]' <<<"$fip")
fip_addr=$(jq -r 'map(.["Floating IP Address"])[0]' <<<"$fip")

### Provisioning GPU nodes

It will take approximately 10 minutes for all resources to be successfully provisioned.

In [ ]:
# Ensure your Jupyter keypair is present
key_pair_upload

sharednet=$(openstack network show sharednet1 -f value -c id)

openstack server create "$RESOURCE_NAME" \
  --flavor baremetal \
  --image "$IMAGE" \
  --nic net-id="$sharednet" \
  --key-name "$USER-jupyter" \
  --hint reservation="$host_reservation_id" \
  --max "$NODE_COUNT"

## Using bare metal nodes

You can log in to the bare metal node as the `cc` user. The SSH authentication should have been set up automatically for you. You can run commands remotely in this notebook, or you can open a new Terminal in JupyterLab to get an interactive session.

In [ ]:
ssh cc@"$fip_addr" sh -c 'Hello from $(hostname)!'

## Release resources

If you finish with your experimentation before your lease expires, it's a good idea to release your resources and tear down your environment to avoid excessive charges against your allocation. You can run the following to tear down the environment (commented out to prevent accidental deletions):

In [ ]:
#blazar lease-delete "$lease_name"